# PSTAT 234 Homework 2 
__Author: Laura Urbisci__  
__Due date: April 25, 2018 by 10 pm__


## Problem 1: Data download

In [7]:
import pandas as pd

def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}
    
    from pandas import DataFrame
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(DataFrame(d, columns=h))

In [208]:
## get all teams
params = {'LeagueID':'00'}
teams = get_nba_data('commonTeamYears', params)

## get all players
params = {'LeagueID':'00', 'Season': '2016-17', 'IsOnlyCurrentSeason': '0'}
players = get_nba_data('commonallplayers', params)

teams.ABBREVIATION = teams.ABBREVIATION.astype('category')
teams.TEAM_ID      = teams.TEAM_ID.astype('category')
teams.MIN_YEAR     = teams.MIN_YEAR.astype('int')
teams.MAX_YEAR     = teams.MAX_YEAR.astype('int')

teams = teams[teams.MAX_YEAR == 2017]
teams['TEAM_AGE'] = teams.MAX_YEAR - teams.MIN_YEAR

teams_clean = teams.copy() ## make a copy for later
team_names = players[['TEAM_ABBREVIATION', 'TEAM_CODE']].drop_duplicates()#.set_index('TEAM_ABBREVIATION')
teams = pd.merge(teams_clean, team_names, left_on='ABBREVIATION', right_on='TEAM_ABBREVIATION')
teams.TEAM_CODE = teams.TEAM_CODE.str.capitalize() # returns values so needs to be reassigned
teams.sort_values('ABBREVIATION', inplace=True)    # modifies object
teams.tail()

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,TEAM_AGE,TEAM_ABBREVIATION,TEAM_CODE
21,00,1610612758,1948,2017,SAC,69,SAC,Kings
22,00,1610612759,1976,2017,SAS,41,SAS,Spurs
24,00,1610612761,1995,2017,TOR,22,TOR,Raptors
25,00,1610612762,1974,2017,UTA,43,UTA,Jazz
27,00,1610612764,1961,2017,WAS,56,WAS,Wizards


## Problem 2: Creating interactive widgets

I decided to create another widget that ....

In [111]:
from ipywidgets import interact, FloatSlider, Dropdown, Button

# First widget from lecture - drinks and types
menu = {
    'juice':['apple', 'peach', 'grape'],
    'tea':['ginger', 'green', 'earl grey'],
}

selected = 'tea'

flavor = Dropdown(options=menu[selected], value=menu[selected][0])
drink = Dropdown(options=menu.keys(), value=selected)
order = Button(description='Order!', icon='check')

def update_drink(change):
    flavor.options = menu[change['new']]
    flavor.selected = menu[change['new']][0]
    
def make_order(change):
    print(flavor.value, drink.value)
    
drink.observe(update_drink, names='value')
order.on_click(make_order)
 
display(flavor, drink, order)

Dropdown(options=('ginger', 'green', 'earl grey'), value='ginger')

Dropdown(index=1, options=('juice', 'tea'), value='tea')

Button(description='Order!', icon='check', style=ButtonStyle())

In [175]:
# second widget from lecture - Teams and players
team_dd_text = teams.TEAM_ABBREVIATION+', '+teams.TEAM_CODE
team_dd = dict(zip(team_dd_text, teams.TEAM_ID))

plyr_by_team_dd = dict()

for t, p in players.groupby('TEAM_ID'):
    plyr_by_team_dd[t] = dict(zip(p.DISPLAY_LAST_COMMA_FIRST, p.PERSON_ID))

plyr_dd_text = players.DISPLAY_LAST_COMMA_FIRST
plyr_dd_id = players.PERSON_ID
plyr_dd = dict(zip(plyr_dd_text, plyr_dd_id))

selected = 'LAC, Clippers'

team_menu = Dropdown(options=team_dd, label=selected)
plyr_menu = Dropdown(options=plyr_by_team_dd[team_dd[selected]])

display(team_menu, plyr_menu)

def update_team(change):
    plyr_menu.options = plyr_by_team_dd[change['new']]

team_menu.observe(update_team, names='value')

Dropdown(index=12, options={'ATL, Hawks': 1610612737, 'BKN, Nets': 1610612751, 'BOS, Celtics': 1610612738, 'CHA, Hornets': 1610612766, 'CHI, Bulls': 1610612741, 'CLE, Cavaliers': 1610612739, 'DAL, Mavericks': 1610612742, 'DEN, Nuggets': 1610612743, 'DET, Pistons': 1610612765, 'GSW, Warriors': 1610612744, 'HOU, Rockets': 1610612745, 'IND, Pacers': 1610612754, 'LAC, Clippers': 1610612746, 'LAL, Lakers': 1610612747, 'MEM, Grizzlies': 1610612763, 'MIA, Heat': 1610612748, 'MIL, Bucks': 1610612749, 'MIN, Timberwolves': 1610612750, 'NOP, Pelicans': 1610612740, 'NYK, Knicks': 1610612752, 'OKC, Thunder': 1610612760, 'ORL, Magic': 1610612753, 'PHI, Sixers': 1610612755, 'PHX, Suns': 1610612756, 'POR, Blazers': 1610612757, 'SAC, Kings': 1610612758, 'SAS, Spurs': 1610612759, 'TOR, Raptors': 1610612761, 'UTA, Jazz': 1610612762, 'WAS, Wizards': 1610612764}, value=1610612746)

Dropdown(options={'Anderson, Alan': 101187, 'Bass, Brandon': 101138, 'Crawford, Jamal': 2037, 'Felton, Raymond': 101109, 'Griffin, Blake': 201933, 'Johnson, Brice': 1627744, 'Johnson, Wesley': 202325, 'Jordan, DeAndre': 201599, 'Mbah a Moute, Luc': 201601, 'Paul, Chris': 101108, 'Pierce, Paul': 1718, 'Redick, JJ': 200755, 'Rivers, Austin': 203085, 'Speights, Marreese': 201578, 'Stone, Diamond': 1627754}, value=101187)

In [214]:
# can i just add on to this add have it display a 3rd thing?
team_age_val = teams.TEAM_AGE
team_age = dict(zip(team_dd_text, team_age_val))

team_age[selected]
#age_menu = Dropdown(options=team_age, label=team_age[selected])
#display(team_menu, plyr_menu, age_menu)



47